In [2]:
import sys,os, pandas as pd,datetime
from pathlib import Path
project_root = os.path.abspath(Path("../."))
sys.path.append(str(project_root))
print(project_root)
from scripts.s2_data_preprocessing import run_data_preprocessing
from scripts.s3_feature_engineering import run_feature_engineering,remove_multicollinear
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

g:\My Drive\0Study_programs\1Msc_ML_AI\EAF_python_project\EAF_implementation_thesis


In [3]:
load_mt5_data=False
extra_symbols= []
base_features=['Close', 'High', 'Log_Return', 'Log_Return_change1', 'Log_Return_change2']


In [4]:
run_data_preprocessing(load_mt5_data, extra_symbols)

Starting data preprocessing
Loading raw data from g:\My Drive\0Study_programs\1Msc_ML_AI\EAF_python_project\EAF_implementation_thesis\data\raw\gold_price_yahoo_20250824.csv
Loading raw data from g:\My Drive\0Study_programs\1Msc_ML_AI\EAF_python_project\EAF_implementation_thesis\data\raw\gold_price_yahoo_20250824.csv
load_raw_data 
                          Close                High                 Low                Open Volume
Date                                                                                             
2005-01-03  428.70001220703125               431.0               431.0               431.0      4
2005-01-04               428.5               428.5               428.5               428.5    108
2005-01-05   426.6000061035156  425.70001220703125  425.70001220703125  425.70001220703125      2
2005-01-06               421.0               421.0               421.0               421.0      1
2005-01-07   418.8999938964844  423.70001220703125  418.70001220703125  423.70

,Close,High,Low,Open,Volume
Date,,,,,
2005-01-03,428.700012,431.000000,431.000000,431.000000,4.0
2005-01-04,428.500000,428.500000,428.500000,428.500000,108.0
2005-01-05,426.600006,425.700012,425.700012,425.700012,2.0
2005-01-06,421.000000,421.000000,421.000000,421.000000,1.0
2005-01-07,418.899994,423.700012,418.700012,423.700012,1.0
...,...,...,...,...,...
2024-12-24,2620.000000,2620.000000,2609.500000,2613.000000,35.0
2024-12-26,2638.800049,2638.800049,2627.899902,2628.500000,84.0
2024-12-27,2617.199951,2617.699951,2616.399902,2617.699951,642.0


In [5]:
# Feature Engineering
for target_isClose in [True,False]:
    features_file = f"gold_price_features_{'Close' if target_isClose else 'LogReturn'}{'_mt5' if load_mt5_data else ''}.csv"
    features_filepath= str(project_root) + f'/data/features/{features_file}'
    run_feature_engineering(
        load_mt5_data,
        base_features=base_features,
        target_isClose=target_isClose,
        features_filepath = features_filepath
    )

    df =pd.read_csv(features_filepath   ,    index_col=0, parse_dates=True)
    features = df.drop(columns=['target'])
    target = df['target']
    print('remove_multicollinear...')
    features_wo_multicolinear = remove_multicollinear(features, threshold=0.9)

    df_OHLCV = features_wo_multicolinear
    df_OHLCV[['Open','High','Low','Close','Volume']] = df[['Open','High','Low','Close','Volume']]
    df_OHLCV['target'] = target
    df_OHLCV.to_csv(
            features_filepath,
            index=True
        )
    df_OHLCV.index.rename('Date', inplace=True)
    if 'Date' in df_OHLCV.columns:
        df_OHLCV['Date'] = pd.to_datetime(df_OHLCV['Date'])
        df_OHLCV.set_index('Date', inplace=True)
    print(df_OHLCV.shape)

Starting feature engineering process
Loading processed data from g:\My Drive\0Study_programs\1Msc_ML_AI\EAF_python_project\EAF_implementation_thesis\data\processed\gold_price_processed.csv
Loading processed data from g:\My Drive\0Study_programs\1Msc_ML_AI\EAF_python_project\EAF_implementation_thesis\data\processed\gold_price_processed.csv
Creating target variable
Creating target variable (next day's closing price)
create_target_variable target_isClose: True
list_Price: ['Close', 'High', 'Log_Return', 'Log_Return_change1', 'Log_Return_change2']
Creating 12 lag features: [1, 2, 3, 5, 10, 20, 30, 60, 90, 120, 180, 250]
Creating rolling features for windows: [5, 10, 20, 30, 60, 90, 180, 250]
Creating 12 lag features: [1, 2, 3, 5, 10, 20, 30, 60, 90, 120, 180, 250]
Creating rolling features for windows: [5, 10, 20, 30, 60, 90, 180, 250]
Creating 12 lag features: [1, 2, 3, 5, 10, 20, 30, 60, 90, 120, 180, 250]
Creating rolling features for windows: [5, 10, 20, 30, 60, 90, 180, 250]
Creating 